In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import random
import math
import time
import sys
import os

sys.path.append(os.path.abspath("./neat"))

from nn import *
from mutation import *
from speciation import *

random.seed(42)
torch.manual_seed(42)

In [21]:
# Init stuff

# Hyperparameters
input_dim = 8*8
output_dim = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
model = torch.load("./models/sklearn_digits_100pop_2000epoch_real96.67.pth", weights_only=False)
# fnn_model = torch.load("./models/fnn97.29.pth", weights_only=False)

In [22]:
import networkx as nx
import matplotlib.pyplot as plt

nodes = [i.id for i in model.nodes]
edges = [(conn.in_node, conn.out_node) for conn in model.connections]

In [23]:
from networkx.algorithms.community import modularity

In [24]:
import networkx as nx

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)


In [26]:
import community as community_louvain
from collections import defaultdict

# Detect best partition
partition = community_louvain.best_partition(G)  # {node: community_id}

# Convert to list of sets
comm_map = defaultdict(set)
for node, comm_id in partition.items():
    comm_map[comm_id].add(node)
communities = list(comm_map.values())

# Now compute modularity
from networkx.algorithms.community import modularity
Q = modularity(G, communities)
print(f"Modularity (Louvain): {Q:.4f}")


Modularity (Louvain): 0.1401


In [27]:
def create_nn_graph(input_size=64, hidden_size=32, output_size=10):
    nodes = []
    edges = []

    # Create node labels
    input_nodes = [f"I{i}" for i in range(input_size)]
    hidden_nodes = [f"H{i}" for i in range(hidden_size)]
    output_nodes = [f"O{i}" for i in range(output_size)]

    # Combine nodes
    nodes.extend(input_nodes)
    nodes.extend(hidden_nodes)
    nodes.extend(output_nodes)

    # Create edges: Fully connect input -> hidden
    for i_node in input_nodes:
        for h_node in hidden_nodes:
            edges.append((i_node, h_node))

    # Create edges: Fully connect hidden -> output
    for h_node in hidden_nodes:
        for o_node in output_nodes:
            edges.append((h_node, o_node))

    return nodes, edges

# Example usage
nodes, edges = create_nn_graph()

In [ ]:
edges

In [28]:
import networkx as nx

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)


In [29]:
import community as community_louvain
from collections import defaultdict

# Detect best partition
partition = community_louvain.best_partition(G)  # {node: community_id}

# Convert to list of sets
comm_map = defaultdict(set)
for node, comm_id in partition.items():
    comm_map[comm_id].add(node)
communities = list(comm_map.values())

# Now compute modularity
from networkx.algorithms.community import modularity
Q = modularity(G, communities)
print(f"Modularity (Louvain): {Q:.4f}")


Modularity (Louvain): 0.0000
